## 基于模式匹配的对话机器人实现

### Pattern Match

机器能否实现对话，这个长久以来是衡量机器人是否具有智能的一个重要标志。 Alan Turing早在其文中就提出过一个测试机器智能程度的方法，该方法主要是考察人类是否能够通过对话内容区分对方是机器人还是真正的人类，如果人类无法区分，我们就称之为具有”智能“。而这个测试，后来被大家叫做”图灵测试“，之后也被翻拍成了一步著名电影，叫做《模拟游戏》。

既然图灵当年以此作为机器是否具备智能的标志，这项任务肯定是复杂的。自从 1960s 开始，诸多科学家就希望从各个方面来解决这个问题，直到如今，都只能解决一部分问题。 目前对话机器人的建立方法有很多，今天的作业中，我们为大家提供一共快速的基于模板的对话机器人配置方式。

```
Pattern: (我想要A)
Response: (如果你有 A，对你意味着什么呢？)

Input: (我想要度假)
Response: (如果你有度假，对你意味着什么呢？)
```

为了实现模板的判断和定义，我们需要定义一个特殊的符号类型，这个符号类型就叫做"variable"， 这个"variable"用来表示是一个占位符。例如，定义一个目标: "I want X"， 我们可以表示成  "I want ?X", 意思就是?X是一个用来占位的符号。

如果输入了"I want holiday"， 在这里 'holiday' 就是 '?X'

### 匹配输入问句的pattern，提取关键信息

In [2]:
import random
import jieba
import re

In [3]:
fail = [True, None]
def pat_match_with_seg(pattern, saying):
    "匹配?*, ?对应位置"
    if not pattern or not saying: return []  # 空
    candidate_p = pattern[0]
    # 匹配一个词
    if is_variable(candidate_p):
        return [(candidate_p, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    # 匹配多个词
    elif is_pattern_segment(candidate_p):
        match, index = segment_match(pattern, saying)
        return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    # 相同继续
    elif candidate_p == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return fail


def is_variable(pat):
    "只能匹配一个词"
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:])


def is_pattern_segment(pattern):
    "匹配多个词"
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:])


def segment_match(pattern, saying):
    "对匹配的第一个部分，进行提取，返回对应位置和saying中末端index"
    seg_pat, rest = pattern[0], pattern[1:]
    seg_pat = seg_pat.replace('?*', '?')

    # 没有rest项，直接匹配saying全部
    if not rest: return (seg_pat, saying), len(saying)

    # 找到匹配符对应的部分
    for i, token in enumerate(saying):
        # rest[0]：首个非匹配符
        if rest[0] == token and is_match(rest[1:], saying[(i + 1):]):
            return (seg_pat, saying[:i]), i

    return (seg_pat, saying), len(saying)


def is_match(rest, saying):
    "判断剩余部分的匹配情况"
    # 空
    if not rest and not saying:
        return True
    # 匹配到匹配符
    if not all(a.isalpha() for a in rest[0]):
        return True
    # 不匹配
    if rest[0] != saying[0]:
        return False
    # 下一个
    return is_match(rest[1:], saying[1:])

In [4]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [5]:
pat_match_with_seg('I need ?*X'.split(), "I need iPhone".split())

[('?X', ['iPhone'])]

### 根据问句信息和答句pattern，生成回答

In [6]:
# 将以上pattern进行替换，输出结果
def pat_trans_dict(pats):
    "格式化匹配结果"
    return {k: ' '.join(v) if isinstance(v, list) else v for k, v in pats}

def substitute(pattern, pats_dict):
    "将用于回答的pattern，利用问句pattern解析的结果，进行处理"
    if not pattern: return []  # 没有答句pattern
    # 从pat_dict中get到pattern，没有就返回自身
    return [pats_dict.get(pattern[0], pattern[0])] \
            + substitute(pattern[1:], pats_dict)

#### 示例

In [7]:
pats_ask = pat_match_with_seg('?*P is very good and ?*X'.split(), \
                              "My dog is very good and my cat is very cute".split())

pat_dict = pat_trans_dict(pats_ask)

answer = substitute('Sorry, you said ?P ?'.split(),  pat_dict)

print(' '.join(answer))

Sorry, you said My dog ?


### Get response

In [9]:
symbol_to_word = {'?*y': "问星外", '?*x': '问星唉', '?*z': "问星贼", '?x': '问唉', '?y': '问外', '?z': '问贼'}
word_to_symbol = {v: k for k, v in symbol_to_word.items()}

for word in word_to_symbol.keys():
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\qw\AppData\Local\Temp\jieba.cache
Loading model cost 0.873 seconds.
Prefix dict has been built succesfully.


In [10]:
def process_pat(pattern):
    "处理symbol划分问题"
    for symbol, word in symbol_to_word.items():
        pattern = pattern.replace(symbol, word)
    pattern_list = [e for e in jieba.cut(pattern)  if e != ' ']
    pattern_list = [word_to_symbol.get(w, w) for w in pattern_list]
    return pattern_list

process_pat('?*x喜欢?*y')

['?*x', '喜欢', '?*y']

In [12]:
def splite_all(process_pat):
    res = []
    for i in process_pat:
        if i in symbol_to_word.keys() or re.match('[A-Za-z]+', i):
            res.append(i)
        else:
            res.extend([e for e in i])
    return res
    
splite_all(process_pat('?*x喜欢AI?*y'))

['?*x', '喜', '欢', 'AI', '?*y']

In [13]:
def get_response(saying, response_rules):
    '获取匹配回答'
    saying = splite_all([e for e in jieba.cut(saying) if e != ' '])

    for ask_pat, ans_pat in response_rules.items():
        ask_pat_p = splite_all(process_pat(ask_pat))
        ask_pat_match = pat_match_with_seg(ask_pat_p, saying)

        ask_pat_match_dict = pat_trans_dict(ask_pat_match)

        if ask_pat_match[0][1] != saying and ask_pat.count('?*') == len(ask_pat_match_dict):
            ans_pat_p = splite_all(process_pat(random.choice(ans_pat)))
            answer = substitute(ans_pat_p, ask_pat_match_dict)
            return answer

    return '我不明白你在说什么'

In [15]:
get_response('我想要泡面', rule_responses)

['你', '可', '以', '想', '想', '你', '很', '快', '就', '可', '以', '要 泡 面', '了']

In [16]:
get_response('小明我很难过，因为很饿', rule_responses)

['很 饿', '不', '应', '该', '让', '你', '这', '么', '难', '过', '的']

### rules

In [14]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

### 问题4

1. 这样的程序有什么优点？有什么缺点？你有什么可以改进的方法吗？ 
2. 什么是数据驱动？数据驱动在这个程序里如何体现？
3. 数据驱动与 AI 的关系是什么？ 


1. 这种程序主要是不灵活，一旦出现规则中不存在的问题，就凉了。因此你需要设计大量的规则，手动处理大量的数据。改进方法在规则的基础上加入相似性匹配算法，相似的问题可以一起处理。然后对于不包含于库中的问题，通过深度学习的方法加以补充。

2. 数据驱动，就是算法会根据已有数据的变化而改变进而取得更好效果。这个程序的数据驱动主要就是由数据而来的规则，以及改变之后的数据的筛选方法的改变。

3. 数据驱动相当于是AI的一个前提条件和一个固有的特征，没有数据AI就只能停留在人工智障阶段，有了数据才有可能提升自身的泛化能力和解决实际问题的能力。